<b><font color="red" size="6">ch03_연관분석</font></b>

- pip install apyori

# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 신상품 카테고리 구성 ...
    
```
{조건 }->{결과}
연관분석과 관련된 지표
1. 지지도(support) : 전체 데이터 중에 조건결과 항목들의 포함된 거래 비율
        (X,Y)의 항목수/ 전체 데이터 수 ex)0.2

2. 신뢰도(confidence): 조건이 발생했을 때, 결과가 동시에 일어날 확률
        (X,Y)의 항목수/ 조건항목수 ex)0.5

3. 향상도(lift): 우연히 발생한 규칙은 아닌지 확인
        조건결과 지지도/ (조건지지도)*(결과지지도)
        <1 : 음의 상관관계
        >1 : 양의 상관관계
                        지지도       신뢰도           향상도
{오렌지주스} ->{와인}    1/5(0.2)    1/2(0.5)    0.2/0.4*0.6(0.833)     

```

In [3]:
%ls data\cf_basket.csv

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: A822-4F57

 D:\ai\source\10_자연어처리\data 디렉터리

2024-12-30  오전 09:03               119 cf_basket.csv
               1개 파일                 119 바이트
               0개 디렉터리  399,558,959,104 바이트 남음


# 2. 연관분석 구현

In [4]:

# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('data/cf_basket.csv','r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    #print(list(csvdata))
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [12]:
from apyori import apriori
rules = apriori(transaction,min_support=0.15,min_confidence=0.1)
rules = list(rules)
rules[15]

RelationRecord(items=frozenset({'맥주', '콜라', '와인'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주', '콜라', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'맥주', '콜라'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '콜라'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [20]:
#items_frozenset({'와인','콜라','맥주'}), support=0.2 : 이 세트가 함께 나타날 확률
#items_base : 조건(lhs)left hand side
#items_add : 결과(rhs)right hand side
row = rules[15]
support = row[1]
ordered_st = row[2]
for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift = item[3]
    if lift>1:
        print("{}=>{} \t {}\t {}\t{}".format(lhs,rhs,support,confidence,lift))
    

맥주=>콜라, 와인 	 0.2	 0.5	1.25
콜라=>맥주, 와인 	 0.2	 0.25	1.25
맥주, 와인=>콜라 	 0.2	 1.0	1.25
콜라, 와인=>맥주 	 0.2	 0.5	1.25


In [32]:
print('조건 \t    -> 결과 \t\t 지지도 신뢰도 향상도')
for row in rules:
        support = row[1]
        ordered_st=row[2]
        for item in ordered_st:
            lhs = ', '.join([x for x in item[0]])
            rhs = ', '.join([x for x in item[1]])
            confidence = item[2]
            lift = item[3]
            if lift>1:
                print("{:10}->{:10} \t {}\t {}\t{}".format(lhs,rhs,support,round(confidence,2),round(lift,2)))

조건 	    -> 결과 		 지지도 신뢰도 향상도
맥주        ->콜라         	 0.4	 1.0	1.25
콜라        ->맥주         	 0.4	 0.5	1.25
소주        ->콜라         	 0.6	 1.0	1.25
콜라        ->소주         	 0.6	 0.75	1.25
콜라        ->소주, 맥주     	 0.2	 0.25	1.25
소주, 맥주    ->콜라         	 0.2	 1.0	1.25
맥주        ->콜라, 와인     	 0.2	 0.5	1.25
콜라        ->맥주, 와인     	 0.2	 0.25	1.25
맥주, 와인    ->콜라         	 0.2	 1.0	1.25
콜라, 와인    ->맥주         	 0.2	 0.5	1.25
소주        ->오렌지주스, 콜라  	 0.2	 0.33	1.67
콜라        ->소주, 오렌지주스  	 0.2	 0.25	1.25
소주, 오렌지주스 ->콜라         	 0.2	 1.0	1.25
오렌지주스, 콜라 ->소주         	 0.2	 1.0	1.67
콜라        ->소주, 와인     	 0.2	 0.25	1.25
소주, 와인    ->콜라         	 0.2	 1.0	1.25


In [37]:
import pandas as pd
rules_df= pd.DataFrame(None,columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in rules:
        support = row[1]
        ordered_st=row[2]
        for item in ordered_st:
            lhs = ', '.join([x for x in item[0]])
            rhs = ', '.join([x for x in item[1]])
            confidence = item[2]
            lift = item[3]
            if lift>1:
                rules_df.loc[index] = [lhs,rhs,support,confidence,lift]
                index +=1
rules_df.sort_values(by= ['confidence','lift'],ascending=False)
                

,lhs,rhs,support,confidence,lift
13,"오렌지주스, 콜라",소주,0.2,1.000000,1.666667
0,맥주,콜라,0.4,1.000000,1.250000
2,소주,콜라,0.6,1.000000,1.250000
5,"소주, 맥주",콜라,0.2,1.000000,1.250000
8,"맥주, 와인",콜라,0.2,1.000000,1.250000
12,"소주, 오렌지주스",콜라,0.2,1.000000,1.250000
15,"소주, 와인",콜라,0.2,1.000000,1.250000
3,콜라,소주,0.6,0.750000,1.250000
1,콜라,맥주,0.4,0.500000,1.250000
6,맥주,"콜라, 와인",0.2,0.500000,1.250000


# 3. 뉴스 연관 분석
- "http://fs.jtbc.joins.com/RSS/economy.xml"-> 기사 검색후 ->명사 추출(기자, 앵커 제외) -> 연관분석(min_support=0.15,min_confidence=0.1)

In [69]:
import requests
from bs4 import BeautifulSoup
rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"

jtbc_economy = requests.get(rss_url)
soup = BeautifulSoup(jtbc_economy.content,'xml')
link_ele= soup.select('item link')
link_list = [link.text for link in link_ele]
print(len(link_list))
link_list

# link_list 들의 뉴스 기사 중 명사만 뽑아 list
from konlpy.tag import Komoran
komoran = Komoran()
news = []
stopwords=['기자','앵커']
for link in link_list:
    response = requests.get(link)
    news_soup = BeautifulSoup(response.content, "html.parser")
    title=news_soup.select_one('title').text
    content = str(news_soup.select_one('meta[name="description"]'))
    content = content.replace('<meta content="', '').replace('" name="description"/>','')
    noun_list = komoran.nouns(title+' '+content)
    #print("nouns 함수 사용:", noun_list)
    noun_list = [ word for word, tag in komoran.pos(title+' ' +content)\
                    if tag in ('NNG','NNP') and len(word)>1]
    filter_noun_list = list(filter(lambda word : word not in stopwords,noun_list))
    news.append(filter_noun_list)
print(news)

rules = apriori(news,min_support=0.15,min_confidence=0.1)
rules = list(rules)
rules


20
[['촌주', '전세', '홍수', '예상', '눈치', '게임', '시작', '뉴스', '단군', '최대', '규모', '재건축', '촌주', '아파트', '입주', '다음', '시작', '규모', '입주', '보니', '전세', '매물도', '현장', '분위기', '이상화', '다음', '세대', '입주', '시작', '서울', '둔촌동', '올림픽', '파크', '포레', '국내', '재건축', '가운데', '역대', '최대', '규모', '이용택', '공인중개사', '공개', '행사', '이후', '판단', '연령', '세대', '관심', '단지', '보니', '입주', '시기', '전세', '물량', '관심', '세대', '입주', '서울', '헬리', '시티', '당시', '전세', '매매', '가격', '동반', '하락', '이번', '현장', '분위기', '김복수', '공인중개사', '일반', '아파트', '입주', '정도', '전세', '시장', '모습', '전세', '입주', '전세', '본인', '가격', '거주', '의무', '폐지', '신축', '선호', '현상', '가세', '전세', '매물', '자체', '부족', '인터넷', '매물', '세대', '최근', '정부', '대출', '관리', '전세', '대출', '규제', '해진', '한몫', '대중', '서강대', '부동산', '학과', '교수', '전세', '대출', '세입자', '물량', '전세', '대출', '입주', '현상', '임대', '물량', '상대', '전세', '집주인', '주변', '전셋값', '신축', '중심', '추세', '영지', '데이터', '규모', '주장', '전세', '가격', '하락', '매물', '유통', '기대', '영상', '취재', '승규', '영상', '편집', '김지우'], ['실제', '보니', '완전', '사기', '캠핑', '관리', '소홀', '뉴스', '캠핑장', '예약', '예약', '플랫폼', '사진', '기대', '보니'

[RelationRecord(items=frozenset({'가격'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'가격'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'내년'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'내년'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'뉴스'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'뉴스'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'분기'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'분기'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'사실'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'사실'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'서울'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozen

In [73]:
rules = apriori(news,min_support=0.15,min_confidence=0.1)
rules = list(rules)
rules


[RelationRecord(items=frozenset({'가격'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'가격'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'내년'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'내년'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'뉴스'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'뉴스'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'분기'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'분기'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'사실'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'사실'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'서울'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozen

In [74]:
import pandas as pd
rules_df= pd.DataFrame(None,columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in rules:
        support = row[1]
        ordered_st=row[2]
        for item in ordered_st:
            lhs = ', '.join([x for x in item[0]])
            rhs = ', '.join([x for x in item[1]])
            confidence = item[2]
            lift = item[3]
            if lift>1:
                rules_df.loc[index] = [lhs,rhs,support,confidence,lift]
                index +=1
rules_df.sort_values(by= ['confidence','lift'],ascending=False)

,lhs,rhs,support,confidence,lift
2,사실,업체,0.15,1.00,6.666667
3,업체,사실,0.15,1.00,6.666667
8,사실,"뉴스, 업체",0.15,1.00,6.666667
9,업체,"뉴스, 사실",0.15,1.00,6.666667
10,"뉴스, 사실",업체,0.15,1.00,6.666667
11,"뉴스, 업체",사실,0.15,1.00,6.666667
0,가격,최근,0.15,0.75,3.750000
1,최근,가격,0.15,0.75,3.750000
4,가격,"뉴스, 최근",0.15,0.75,3.750000
5,최근,"뉴스, 가격",0.15,0.75,3.750000
